In [2]:
from openai import OpenAI
client = OpenAI()

In [7]:
# Upload a file with an "assistants" purpose
files = ['contacts.csv', 'searches.csv']
uploaded_files = [] 

# for f in files:
#   file = client.files.create(
#     file=open(f"../data/{f}", "rb"),
#     purpose='assistants'
#   )
#   uploaded_files.append(file)

# file_ids = [file.id for file in uploaded_files]

file_ids = ['file-4FKpqlvJOZqxNUfIJHAHRGJY', 'file-TzOVAHko5TwyrRsaPySmIR0Z']
file_ids



['file-4FKpqlvJOZqxNUfIJHAHRGJY', 'file-TzOVAHko5TwyrRsaPySmIR0Z']

In [8]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="You are a personal chatbot assistant.",
  model="gpt-4-turbo",
  tools=[{"type": "code_interpreter"}],
  file_ids=file_ids
)

In [4]:
thread = client.beta.threads.create()

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="which day of the week were the most bookings made?"
)

In [14]:
message

Message(id='msg_65lBESnXqLnniNHAI865s36W', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='which day of the week were the most bookings made?'), type='text')], created_at=1713076608, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_xkjf4vsQK6FTO68FKuSAmhk2')

In [15]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `create_and_stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > To help you find which day of the week had the most bookings, I'll need to know the format and type of data in the files you've uploaded. Let's start by examining the contents of the files to understand the structure of the data, including how dates are recorded. I will then extract the necessary date information to assess on which day of the week the most bookings were made. Let me first check the content of the files you provided.
assistant > code_interpreter

import pandas as pd

# Load the files to see their content and format
file_path1 = '/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY'
file_path2 = '/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z'

# Since we don't know the file types, we'll try to guess by file extensions
file_path1.split('.')[-1], file_path2.split('.')[-1]

output >

('/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY',
 '/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z')

assistant > It seems there was an issue determining the file types as the extensions weren't visible in the pa